In [3]:
import os
import torch
import tensorflow as tf
import numpy as np
import pandas as pd
import random
import time
import datetime

from joblib import load
from collections import namedtuple
from typing import List, Dict, Tuple
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, random_split

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from joblib import dump, load

import transformers 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, BertConfig, get_linear_schedule_with_warmup

In [4]:
#Get the GPU device name.
device_name = tf.test.gpu_device_name()
#The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print("GPU device not found")
    #raise SystemError('GPU device not found')

# If there's a GPU available...
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

GPU device not found
No GPU available, using the CPU instead.


In [5]:
df = pd.read_csv('final_df.csv')
df_fiction = df[['book','genres_Fiction']]

In [6]:
df_fiction['genres_Fiction'].value_counts()

True     31313
False    30139
Name: genres_Fiction, dtype: int64

In [7]:
#import bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

In [8]:
print ('Original: ', df_fiction['book'][0])

Original:  Battle Royale, Vol. 14 The beginning of the end!Shuuya, Kawada, and Noriko fight for their lives against Kiriyama, the psychotic student who decimated most of their class. While Kiriyama strives to eradicate the other kids' futures, we get an insider's view of Kiriyama's past... and what forces shaped this bright, talented young man into the unfeeling killer he is today.
The beginning of the end!Shuuya, Kawada, and Noriko fight for their lives against Kiriyama, the psychotic student who decimated most of their class. While Kiriyama strives to eradicate the other kids' futures, we get an insider's view of Kiriyama's past... and what forces shaped this bright, talented young man into the unfeeling killer he is today.
...more


In [9]:
print ('Tokenized: ', tokenizer.tokenize(df_fiction['book'][0]))

Tokenized:  ['battle', 'royale', ',', 'vol', '.', '14', 'the', 'beginning', 'of', 'the', 'end', '!', 'shu', '##uy', '##a', ',', 'ka', '##wad', '##a', ',', 'and', 'nor', '##iko', 'fight', 'for', 'their', 'lives', 'against', 'ki', '##ri', '##yama', ',', 'the', 'psychotic', 'student', 'who', 'dec', '##imated', 'most', 'of', 'their', 'class', '.', 'while', 'ki', '##ri', '##yama', 'strive', '##s', 'to', 'era', '##dicate', 'the', 'other', 'kids', "'", 'futures', ',', 'we', 'get', 'an', 'insider', "'", 's', 'view', 'of', 'ki', '##ri', '##yama', "'", 's', 'past', '.', '.', '.', 'and', 'what', 'forces', 'shaped', 'this', 'bright', ',', 'talented', 'young', 'man', 'into', 'the', 'un', '##fe', '##elin', '##g', 'killer', 'he', 'is', 'today', '.', 'the', 'beginning', 'of', 'the', 'end', '!', 'shu', '##uy', '##a', ',', 'ka', '##wad', '##a', ',', 'and', 'nor', '##iko', 'fight', 'for', 'their', 'lives', 'against', 'ki', '##ri', '##yama', ',', 'the', 'psychotic', 'student', 'who', 'dec', '##imated', 'm

In [10]:
#print the sentence mapped to token ids
print ('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df_fiction['book'][0])))

Token IDs:  [2645, 24483, 1010, 5285, 1012, 2403, 1996, 2927, 1997, 1996, 2203, 999, 18454, 26230, 2050, 1010, 10556, 26016, 2050, 1010, 1998, 4496, 12676, 2954, 2005, 2037, 3268, 2114, 11382, 3089, 11613, 1010, 1996, 27756, 3076, 2040, 11703, 20592, 2087, 1997, 2037, 2465, 1012, 2096, 11382, 3089, 11613, 29453, 2015, 2000, 3690, 16467, 1996, 2060, 4268, 1005, 17795, 1010, 2057, 2131, 2019, 25297, 1005, 1055, 3193, 1997, 11382, 3089, 11613, 1005, 1055, 2627, 1012, 1012, 1012, 1998, 2054, 2749, 5044, 2023, 4408, 1010, 10904, 2402, 2158, 2046, 1996, 4895, 7959, 18809, 2290, 6359, 2002, 2003, 2651, 1012, 1996, 2927, 1997, 1996, 2203, 999, 18454, 26230, 2050, 1010, 10556, 26016, 2050, 1010, 1998, 4496, 12676, 2954, 2005, 2037, 3268, 2114, 11382, 3089, 11613, 1010, 1996, 27756, 3076, 2040, 11703, 20592, 2087, 1997, 2037, 2465, 1012, 2096, 11382, 3089, 11613, 29453, 2015, 2000, 3690, 16467, 1996, 2060, 4268, 1005, 17795, 1010, 2057, 2131, 2019, 25297, 1005, 1055, 3193, 1997, 11382, 3089, 116

In [13]:
max_len = 0
for i in range(len(df_fiction)):
  #Tokenize the text and add '[CLS]' and '[SEP]' tokens
  synopsis = tokenizer.encode(df_fiction['book'][i], add_special_tokens = True)
  max_len = max(max_len, len(synopsis))
print ('Max sentence length: ', max_len)

Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


Max sentence length:  4387


In [11]:
input_ids = []
attention_masks = []
labels = df_fiction['genres_Fiction']
max_len = 512
#For every sentence
for i in range(len(df_fiction)):
  encoded_dict = tokenizer.encode_plus(
      df_fiction['book'][i],
      add_special_tokens = True, #add '[CLS]' and '[SEP]'
      max_length = max_len, #pad and truncate 
      pad_to_max_length = True,
      return_attention_mask = True, #construct attention mask
      return_tensors = 'pt', #return pytorch tensors
  )

  #add the encoded sentence to the list
  input_ids.append(encoded_dict['input_ids'])

  #add its attention mask (simply differentiates padding from non-padding)
  attention_masks.append(encoded_dict['attention_mask'])

#convert the lists into tensors
input_ids = torch.cat(input_ids, dim = 0)
attention_masks = torch.cat(attention_masks, dim = 0)
labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
#combine the training inputs into a tensor dataset
dataset = TensorDataset(input_ids, attention_masks, labels)

train_size = int(0.8*len(dataset))
val_size = len(dataset) - train_size

#divide the dataset by randomly selecting samples
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

49,161 training samples
12,291 validation samples


In [13]:
batch_size = 32
train_dataloader = DataLoader(
    train_dataset,
    sampler = RandomSampler(train_dataset), #select batch randomly
    batch_size = batch_size
)

#for validation, the order doesn't matter, so just read them sequentially
validation_dataloader = DataLoader(
    val_dataset,
    sampler = SequentialSampler(val_dataset),
    batch_size = batch_size
)

In [14]:
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", #use the 12-layer bert model with an uncased vocab
    num_labels = 2, #for binary classification
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [15]:
optimizer = AdamW(model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 )

In [16]:
epochs = 4
total_steps = len(train_dataloader) * epochs

#create the learning rate scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [17]:
def flat_accuracy(preds, labels):
  pred_flat = np.argmax(preds, axis = 1).flatten()
  labels_flat = labels.flatten()
  return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [18]:
def format_time(elapsed):
  elapsed_rounded = int(round(elapsed))
  return str(datetime.timedelta(seconds = elapsed_rounded))

In [ ]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
#torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        #model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        #print(output[1])
        
        #loss = loss_fn(logits, b_labels)
        
        loss = output.loss
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    best_eval_accuracy = np.Inf
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0]
        b_input_mask = batch[1]
        b_labels = batch[2]
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        loss = output.loss
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = output.logits
        logits = logits.detach().numpy()
        label_ids = b_labels.numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    if avg_val_accuracy < best_eval_accuracy:
        
        torch.save(model, 'bert_model')
        best_eval_accuracy = avg_val_accuracy
    #print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    #print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...


In [ ]:
model = torch.load("bert_model")